# Setup

Link to data in Drive:
https://drive.google.com/drive/folders/1eVA-xVbFa1VDfJ1q_Iu52vFUdx1CsTFx?usp=sharing

In [0]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#os.listdir('/content/gdrive/My Drive/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Read and organize data

In [0]:
markers_path = '/content/gdrive/My Drive/anyway_tables_csv_updated/markers_hebrew.csv'
markers_df = pd.read_csv(markers_path)

In [0]:
tabmef = pd.read_csv('/content/gdrive/My Drive/anyway_tables_csv_updated/Traffic/2017/תקליטור/h20171092tabmef.csv')

In [0]:
markers_df = pd.read_csv(markers_path)
markers_df_selected = markers_df.sort_values(by=['road1'])
wanted_fields = ['provider_and_id','road1', 'km', 'accident_severity_hebrew', 'location_accuracy', 'multi_lane_hebrew']
markers_df_selected = markers_df_selected[wanted_fields] #.query('location_accuracy==1')
clean_markers = markers_df_selected.dropna(subset=['road1','km'])

tabmef = pd.read_csv('/content/gdrive/My Drive/anyway_tables_csv_updated/Traffic/2017/תקליטור/h20171092tabmef.csv')
tabmef = tabmef.drop_duplicates(['kvish','keta','hodesh','taarich','shaa']) # spurious duplicate entries
traffic_by_keta = tabmef.groupby(['kvish','keta','maslul'])['nefah'].mean().reset_index().groupby(['kvish','keta'])['nefah'].sum().reset_index()

traffic_keys = pd.read_csv('/content/gdrive/My Drive/anyway_tables_csv_updated/Traffic/2017/תקליטור/roads_non_urban_no_unicode_real_2.csv', encoding='utf-8')
merged_traffic_info = traffic_keys.merge(traffic_by_keta, left_on=['road', 'keta'], right_on=['kvish', 'keta'])
import sqlite3
connection = sqlite3.connect(':memory:')
merged_traffic_info.to_sql('traffic', connection)
clean_markers.to_sql('markers', connection)
accidents_and_traffic = pd.read_sql_query("""
SELECT *
FROM markers left outer join traffic
on (markers.road1 = traffic.road
AND traffic.km_from * 10 <= markers.km and traffic.km_to * 10 >= markers.km
AND location_accuracy=1)
""", connection)

In [0]:
print(markers_df_selected['road1'].notnull().value_counts())
print(len(clean_markers))
print(clean_markers['road1'].isin(traffic_keys['road'].unique()).value_counts())
print(accidents_and_traffic['km_from'].notnull().value_counts())
save_columns = ['provider_and_id','road1','km','km_from','km_to','keta','nefah']
accidents_and_traffic[save_columns].to_csv('/content/gdrive/My Drive/anyway_tables_csv_updated/2017-accidents_and_traffic.csv')

In [0]:
# stats about which segments we have traffic data for
traffic_keys['keta_length'] = traffic_keys['km_to']-traffic_keys['km_from']
traffic_keys.groupby('road')['keta_length'].sum() / traffic_keys.groupby('road')['km_to'].max()
traffic_keys.groupby('road')['keta_length'].sum().sum() / traffic_keys.groupby('road')['km_to'].max().sum()

0.5156099190734279

# Normalize accidents by traffic

In [0]:
accidents_and_traffic_merged = accidents_and_traffic.merge(markers_df[['provider_and_id','accident_severity']],on='provider_and_id')
accidents_and_traffic_merged['n_accidents'] = 1
accidents_and_traffic_merged['n_accidents_serious'] = (accidents_and_traffic_merged['accident_severity']<=2).astype(int)
accidents_and_traffic_merged['n_accidents_fatal'] = (accidents_and_traffic_merged['accident_severity']==1).astype(int)

In [0]:
def get_risk(accidents_and_traffic_merged, cost_col='n_accidents', plotting=False):
  by_keta = accidents_and_traffic_merged.dropna(subset=['km_to']).groupby(['road1','keta','km_from','km_to', 'from_intersection', 'to_intersection'])['nefah'].mean().to_frame()
  by_keta['cost'] = accidents_and_traffic_merged.dropna(subset=['km_to']).groupby(['road1','keta','km_from','km_to', 'from_intersection', 'to_intersection'])[cost_col].sum()
  by_keta['multi_lane_hebrew'] = accidents_and_traffic_merged.dropna(subset=['km_to']).groupby(['road1','keta','km_from','km_to', 'from_intersection', 'to_intersection'])['multi_lane_hebrew'].agg(lambda x:x.value_counts().index[0] if len(x.value_counts().index) else None)
  by_keta.reset_index(inplace=True)
  by_keta['keta_length'] = by_keta['km_to'] - by_keta['km_from']
  by_keta['risk_per_traffic_km'] = by_keta.eval('cost / nefah / keta_length')
  by_keta['risk_per_traffic'] = by_keta.eval('cost / nefah')
  by_keta.sort_values('risk_per_traffic')[::-1]
  by_keta['log_nefah'] = np.log1p(by_keta['nefah'])
  by_keta['log_cost'] = np.log1p(by_keta['cost'])
  by_keta['log_nefah_km'] = np.log1p(by_keta['nefah']*by_keta['keta_length'])
  if plotting:
    by_keta.plot.scatter('log_nefah_km','log_cost')
    plt.title(cost_col)
    plt.show()
    #plt.savefig('/content/gdrive/My Drive/traffic_results/{}_graph1'.format(cost_col))
    sns.lmplot('log_nefah_km','log_cost',data=by_keta)
    plt.title(cost_col)
    plt.show()
    #plt.savefig('/content/gdrive/My Drive/traffic_results/{}_graph2'.format(cost_col))
    print(by_keta[['nefah','log_nefah','log_nefah_km','cost','log_cost']].corr())
    #by_keta[['nefah','log_nefah','log_nefah_km','cost','log_cost']].corr().to_csv('/content/gdrive/My Drive/traffic_results/{}_corrcoefs.csv'.format(cost_col))
  by_keta.sort_values('risk_per_traffic_km')[::-1]
  years_in_accident_data = 2018-2008+1 # markers_df['accident_year'].nunique()
  hours_in_year = 24*365
  by_keta['risk_per_km'] = by_keta['risk_per_traffic_km']/(years_in_accident_data*hours_in_year)
  # markers_df['accident_year'].value_counts()
  by_keta.sort_values('risk_per_km',ascending=False,inplace=True)
  by_keta['rank_cost'] = by_keta['cost'].rank(ascending=False)
  by_keta['rank_risk_per_km'] = by_keta['risk_per_km'].rank(ascending=False)
  if plotting:
    plt.figure()
    sns.distplot(by_keta['risk_per_traffic_km'])
    #plt.title(cost_col)
    plt.show()
    #plt.savefig('/content/gdrive/My Drive/traffic_results/{}_graph3'.format(cost_col))
  return by_keta





In [0]:
by_keta_accidents = get_risk(accidents_and_traffic_merged,'n_accidents', True)
by_keta_serious = get_risk(accidents_and_traffic_merged,'n_accidents_serious', False)
by_keta_fatal = get_risk(accidents_and_traffic_merged,'n_accidents_fatal', False)

# Check if dangerous intersections have Mifrada

In [0]:
by_keta_accidents.sort_values('risk_per_km',ascending=False)[['road1', 'keta', 'from_intersection', 'to_intersection', 'cost','multi_lane_hebrew','risk_per_traffic_km']].rename(columns={'cost': 'num_accidents'})

,road1,keta,from_intersection,to_intersection,num_accidents,multi_lane_hebrew,risk_per_traffic_km
8,3.0,20.0,צומת טוביה,צומת מלאכי (קסטינה),169,מיפרדה בנויה ללא גדר בטיחות,0.490009
144,70.0,90.0,מחלף סומך,צומת אבליים,486,מיפרדה בנויה ללא גדר בטיחות,0.443245
6,3.0,10.0,צומת אבא הלל,צומת הודיה,300,מיפרדה בנויה ללא גדר בטיחות,0.286870
127,65.0,30.0,צומת חדרה (מזרח),צומת אלון (שמורת אלון),291,מיפרדה עם גדר בטיחות,0.285086
19,4.0,30.0,צומת ברכיה,צומת אבא הלל,290,מיפרדה בנויה ללא גדר בטיחות,0.262901
16,4.0,10.0,צומת חבל עזה,צומת יד מרדכי,119,מיפרדה בנויה ללא גדר בטיחות,0.259723
129,65.0,50.0,צומת חנה,צומת מנשה,229,מיפרדה עם גדר בטיחות,0.254538
146,70.0,110.0,צומת יבור,צומת אחיהוד,602,מיפרדה בנויה ללא גדר בטיחות,0.250208
169,85.0,26.0,צומת כרמיאל מערב,צומת כרמיאל,410,מיפרדה עם גדר בטיחות,0.249607
72,38.0,20.0,צומת האלה,כניסה לרמת בית שמש (דרום),534,מיפרדה בנויה ללא גדר בטיחות,0.239625
